In [4]:
import h5py
import numpy as np
import keras
import TextCNN
import TextCNN1
import TextCNNK2
import TextRNN
import TextRNN_LSTM
import TextRCNN
import pandas as pd

np.random.seed(0)
content_matrix=pd.read_csv('data/word2vec/content_matrix.csv',header=None)
title_matrix=pd.read_csv('data/word2vec/title_matrix.csv',header=None)
train_data=h5py.File('data/train/train_data.h5','r')
train_title = train_data['train_title'].value
train_content = train_data['train_content'].value
train_label=train_data['train_label'].value

# split_index=np.arange(train_label.shape[0])
# np.random.shuffle(split_index)
# train_index=split_index[0:int(len(split_index)*0.8)]
# eval_index=split_index[int(len(split_index)*0.8):]

# X_eval_title=train_title[eval_index,:]
# X_eval_content=train_content[eval_index,:]
# y_eval=train_label[eval_index]
# X_train_title=train_title[train_index,:]
# X_train_content=train_content[train_index,:]
# y_train=train_label[train_index]
# train_data.close()
# NLP_model=TextCNN.TextCNN(title_matrix=title_matrix,content_matrix=content_matrix)
# NLP_model.model.fit([X_train_title,X_train_content], y_train, batch_size=256, epochs=30, validation_data=([X_eval_title,X_eval_content],y_eval),
#                     callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto'),
#                     keras.callbacks.ModelCheckpoint('model_weights/NLP_model_CNN256.h5', monitor='val_acc', verbose=0, save_best_only=True,
#                                                     save_weights_only=False, mode='auto', period=1)])

#CNN 18epoches 0.84716
#RNN+GRU 8epoches 0.87725
#RNN+LSTM 12epoches 0.84475
#CNN512+RNN 0.89383
#CNN256+RNN 0.89118
#RCNN 21epoches

#Bbang
#RCNN+RNN+CNN256 0.87036

In [5]:
test_data=h5py.File('data/eval/eval_data.h5','r')
test_title = test_data['eval_title'].value
test_content = test_data['eval_content'].value
test_id=test_data['eval_id'].value
test_id=np.array(list(map(lambda x:str(x,encoding = "utf-8"),test_id)))

NLP_model=TextRCNN.TextRCNN(title_matrix=title_matrix,content_matrix=content_matrix)
NLP_model.model.load_weights('model_weights/NLP_model_RCNN.h5')
preds1=NLP_model.model.predict([test_title,test_content],batch_size=256,verbose=1)
result=pd.DataFrame([test_id,preds1],index=['id','pred'])
result=result.transpose()
result.to_csv('model_results/RCNN_prob.csv',index=False,header=False)

NLP_model=TextCNNK2.TextCNNK2(title_matrix=title_matrix,content_matrix=content_matrix)
NLP_model.model.load_weights('model_weights/NLP_model_CNNK2.h5')
preds2=NLP_model.model.predict([test_title,test_content],batch_size=256,verbose=1)
result=pd.DataFrame([test_id,preds2],index=['id','pred'])
result=result.transpose()
result.to_csv('model_results/CNNK2_prob.csv',index=False,header=False)

NLP_model=TextCNN1.TextCNN1(title_matrix=title_matrix,content_matrix=content_matrix)
NLP_model.model.load_weights('model_weights/NLP_model_CNN512.h5')
preds3=NLP_model.model.predict([test_title,test_content],batch_size=256,verbose=1)
result=pd.DataFrame([test_id,preds3],index=['id','pred'])
result=result.transpose()
result.to_csv('model_results/CNN512_prob.csv',index=False,header=False)

NLP_model=TextRNN.TextRNN(title_matrix=title_matrix,content_matrix=content_matrix)
NLP_model.model.load_weights('model_weights/NLP_model_RNN.h5')
preds4=NLP_model.model.predict([test_title,test_content],batch_size=256,verbose=1)
result=pd.DataFrame([test_id,preds4],index=['id','pred'])
result=result.transpose()
result.to_csv('model_results/RNN_prob.csv',index=False,header=False)
preds=(preds1+preds2+preds3+preds4)/4

result=pd.DataFrame([test_id,preds],index=['id','pred'])
result=result.transpose()
result.loc[:,'result']='NEGATIVE'
result.loc[(result['pred']>=0.5),'result']='POSITIVE'
result.drop('pred',axis=1,inplace=True)
result.to_csv('result.csv',index=False,header=False)

400000/400000 [==============================] - 1963s 5ms/step


In [4]:
#多模型直接融合
import pandas as pd
import numpy as np
result_RCNN=pd.read_csv('model_results/RCNN_prob.csv',header=None)
result_CNNK2=pd.read_csv('model_results/CNNK2_prob.csv',header=None)
result_CNN512=pd.read_csv('model_results/CNN512_prob.csv',header=None)
result_RNN=pd.read_csv('model_results/RNN_prob.csv',header=None)

In [5]:
result_RNN.head()

,0,1
0,d64d0c87e3c51019b922df146f8b7eab,[ 0.0053655]
1,8f435a5f65eb0a44523d39b656cfc265,[ 0.99967933]
2,af6b2aac5d7dbd9a868bc70d7b5159e2,[ 0.01040022]
3,18dc0d60fe1f0fbdee72b4c66a00b43e,[ 0.9985252]
4,a69e7fd24cd23da638f95ca5713dceb2,[ 0.99975568]


In [6]:
result_RCNN.head()

,0,1
0,d64d0c87e3c51019b922df146f8b7eab,[ 1.13385486e-05]
1,8f435a5f65eb0a44523d39b656cfc265,[ 0.94399279]
2,af6b2aac5d7dbd9a868bc70d7b5159e2,[ 0.00093789]
3,18dc0d60fe1f0fbdee72b4c66a00b43e,[ 0.9049598]
4,a69e7fd24cd23da638f95ca5713dceb2,[ 0.95701653]


In [19]:
final_result=pd.DataFrame()
final_result.loc[:,'id']=result_RNN[0]
final_result.loc[:,'prob']=np.array(list(map(lambda x:float(x.strip('[]')),result_RCNN[1].values)))*0.3\
+np.array(list(map(lambda x:float(x.strip('[]')),result_RNN[1].values)))*0.3\
+np.array(list(map(lambda x:float(x.strip('[]')),result_CNN512[1].values)))*0.2\
+np.array(list(map(lambda x:float(x.strip('[]')),result_CNNK2[1].values)))*0.2
final_result.head()

,id,prob
0,d64d0c87e3c51019b922df146f8b7eab,0.028894
1,8f435a5f65eb0a44523d39b656cfc265,0.982756
2,af6b2aac5d7dbd9a868bc70d7b5159e2,0.089061
3,18dc0d60fe1f0fbdee72b4c66a00b43e,0.960509
4,a69e7fd24cd23da638f95ca5713dceb2,0.985529


In [21]:
final_result.loc[:,'result']='NEGATIVE'
final_result.loc[(final_result['prob']>=0.5),'result']='POSITIVE'
final_result.drop('prob',axis=1,inplace=True)
final_result.to_csv('result.csv',index=False,header=False)